In [71]:
import numpy as np 
import os 
import pandas as pd
import json
from tqdm import tqdm

base_result_path = "/opt/ml/input/code/predictions"

In [72]:
submission_files = os.listdir(base_result_path)
print(submission_files)

['0.96180.csv', '0.9577.csv', '0.9477.csv', '0.9537.csv', 'output.csv']


In [73]:
from collections import defaultdict
anno_list = defaultdict(list)

for file in submission_files:
    file_path = os.path.join(base_result_path,file)
    
    with open(file_path, 'r') as f:
        anno = json.load(f)
    file_list = sorted(anno['images'].keys())
    for i in file_list:
        bboxes = []
        for word_info in anno['images'][i]['words'].values():
            bboxes.append(np.array(word_info['points']))
        anno_list[i].append(np.array(bboxes))

print(anno_list.keys())
print(len(anno_list['drp.en_ko.in_house.deepnatural_001892.jpg']))



dict_keys(['drp.en_ko.in_house.deepnatural_001892.jpg', 'drp.en_ko.in_house.deepnatural_002413.jpg', 'drp.en_ko.in_house.deepnatural_002415.jpg', 'drp.en_ko.in_house.deepnatural_002420.jpg', 'drp.en_ko.in_house.deepnatural_002436.jpg', 'drp.en_ko.in_house.deepnatural_002442.jpg', 'drp.en_ko.in_house.deepnatural_002458.jpg', 'drp.en_ko.in_house.deepnatural_002460.jpg', 'drp.en_ko.in_house.deepnatural_002492.jpg', 'drp.en_ko.in_house.deepnatural_002499.jpg', 'drp.en_ko.in_house.deepnatural_002517.jpg', 'drp.en_ko.in_house.deepnatural_002518.jpg', 'drp.en_ko.in_house.deepnatural_002523.jpg', 'drp.en_ko.in_house.deepnatural_002535.jpg', 'drp.en_ko.in_house.deepnatural_002538.jpg', 'drp.en_ko.in_house.deepnatural_002576.jpg', 'drp.en_ko.in_house.deepnatural_002578.jpg', 'drp.en_ko.in_house.deepnatural_002620.jpg', 'drp.en_ko.in_house.deepnatural_002623.jpg', 'drp.en_ko.in_house.deepnatural_002626.jpg', 'drp.en_ko.in_house.deepnatural_002631.jpg', 'drp.en_ko.in_house.deepnatural_002633.jpg',

In [74]:
def bb_intersection_over_union(boxA, boxB):
    # 각 박스의 좌표를 추출합니다.
    xA1, yA1, xA3, yA3 = boxA[0][0], boxA[0][1], boxA[2][0], boxA[2][1]
    xB1, yB1, xB3, yB3 = boxB[0][0], boxB[0][1], boxB[2][0], boxB[2][1]

    # 겹치는 영역의 좌표를 계산합니다.
    xA = max(xA1, xB1)
    yA = max(yA1, yB1)
    xB = min(xA3, xB3)
    yB = min(yA3, yB3)

    # 겹치는 영역의 넓이를 계산합니다.
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)

    # 두 박스의 넓이를 계산합니다.
    boxAArea = (xA3 - xA1 + 1) * (yA3 - yA1 + 1)
    boxBArea = (xB3 - xB1 + 1) * (yB3 - yB1 + 1)

    # IoU를 계산합니다.
    iou = interArea / float(boxAArea + boxBArea - interArea)

    # IoU를 반환합니다.
    return iou

In [75]:
num_files = len(submission_files)
ufo_result = dict(images=dict())

for key in tqdm(anno_list.keys()):
    # 한 이미지당 bbox
    base = anno_list[key][0]
    # print(f"base shape: {np.array(base).shape}")
    for points in anno_list[key]:
        if base.shape[0] < points.shape[0]:
            base = points
    new_points = []
    
    # 기본 box 순회 (4,2)
    for base_point in base:
        average_list = []
        # 각 이미지당 bbox 순회 (n,4,2)
        for points in anno_list[key]:
            best_iou = 0
            best_point = None
            for point in points:
                iou = bb_intersection_over_union(base_point,point)
                if iou > 0.3 and iou > best_iou :
                    best_iou = iou
                    best_point = point
                    
            if best_point is not None:
                average_list.append(best_point)
        # average_list [n,4,2]
        if len(average_list) > (num_files//2):
            new_p = np.mean(np.array(average_list),axis=0)
            new_points.append(new_p)
            
    # print(f"new_point shape :{np.array(new_points).shape}")
    words_info = {idx: dict(points=bbox.tolist()) for idx, bbox in enumerate(new_points)}
    ufo_result['images'][key] = dict(words=words_info)

    

100%|██████████| 100/100 [04:45<00:00,  2.86s/it]


In [76]:
output_fname = 'ensemble.csv'
with open(os.path.join("/opt/ml/input/code/predictions/", output_fname), 'w') as f:
    json.dump(ufo_result, f, indent=4)